In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
np.random.seed(18069015) 

In [3]:
# Shipment cost from factory to distribution center
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 12),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (4,3))
CostFtoD = pd.DataFrame(RandomFtoD2, columns=['Atlas', 'Nebule',"Azure"], 
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoD # gives the shipment cost from factory to distribution center

In [4]:
# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (3,5))
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"], 
                        index=['Atlas', 'Nebule',"Azure"])
#CostDtoS # gives the shipment cost from distribution center to store

In [5]:
# Shipment cost from factory to store
RandomFtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 20),0).astype(int)
RandomFtoS2 = np.reshape(RandomFtoS1, (4,5))
CostFtoS = pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoS # gives the shipment cost from distribution center to store

In [6]:
# Your Required Information About Shipment Cost
print("The shipment cost from factory to distribution center")
print(CostFtoD)
print("\nThe shipment cost from distribution center to store")
print(CostDtoS)
print("\nThe shipment cost from factory to store")
print(CostFtoS)

The shipment cost from factory to distribution center
             Atlas  Nebule  Azure
Tonoz            5       7      8
White Kiosk     10       2      6
Quarterage       4       2      3
Middle Yard      5       8      3

The shipment cost from distribution center to store
        Venus  Minerva  Neptunus  Mars  Ceres
Atlas       3        9         1     5      7
Nebule      8        5         4     2      3
Azure       7       10         3     6      8

The shipment cost from factory to store
             Venus  Minerva  Neptunus  Mars  Ceres
Tonoz            3        6         8     1      7
White Kiosk      9        5         7     5      3
Quarterage       7        4         3     1      1
Middle Yard      1        3         2     2      2


In [7]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pulp import*

In [9]:
# definition of sets and parameters

# Factories, distribution centers and stores (sets)
Factory = ["Tonoz","White Kiosk","Quarterage","Middle Yard"]
DistributionCenter = ['Atlas', 'Nebule',"Azure"]
Stores = ["Venus", "Minerva", "Neptunus", "Mars", "Ceres"]

# Capacities
FCapacity = {"Tonoz":3000,
             "White Kiosk":5000,
             "Quarterage":10000,
             "Middle Yard":7000}

# Demand of the stores
Demand = {"Venus":5500, 
          "Minerva":4750,
          "Neptunus":6550, 
          "Mars":4000, 
          "Ceres":4200}

# Costs
CostFDC = {"Tonoz":{'Atlas':5, 
                    'Nebule':7,
                    "Azure":8},
           "White Kiosk":{'Atlas':10, 
                    'Nebule':2,
                    "Azure":6},
           "Quarterage":{'Atlas':4, 
                    'Nebule':2,
                    "Azure":3},
           "Middle Yard":{'Atlas':5, 
                    'Nebule':8,
                    "Azure":3}}
           
CostDCS = {'Atlas':{"Venus":3, "Minerva":9, "Neptunus":1, "Mars":5, "Ceres":7}, 
           'Nebule':{"Venus":8, "Minerva":5, "Neptunus":4, "Mars":2, "Ceres":3},
           "Azure":{"Venus":7, "Minerva":10, "Neptunus":3, "Mars":6, "Ceres":8}}


CostFS = {"Tonoz":{"Venus":3, "Minerva":6, "Neptunus":8, "Mars":1, "Ceres":7},
           "White Kiosk":{"Venus":9, "Minerva":5, "Neptunus":7, "Mars":5, "Ceres":3},
           "Quarterage":{"Venus":7, "Minerva":4, "Neptunus":3, "Mars":1, "Ceres":1},
           "Middle Yard":{"Venus":1, "Minerva":3, "Neptunus":2, "Mars":2, "Ceres":2}}


In [10]:
prob = LpProblem("YTUManufacturing", LpMinimize)

### Decision variable

In [11]:
X = LpVariable.dicts("x",
                     (Factory, DistributionCenter),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Y = LpVariable.dicts("x",
                     (DistributionCenter, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)
Z= LpVariable.dicts("x",
                     (Factory, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

### Objective function

In [12]:
prob += lpSum(CostFDC[i][j]*X[i][j] \
              for i in Factory for j in DistributionCenter) + lpSum(CostDCS[j][k]*Y[j][k] \
                                                                    for j in DistributionCenter for k in Stores)+ lpSum(CostFS[i][k]*Z[i][k]\
                                                                                                                        for i in Factory for k in Stores)


### Constraints

In [13]:
# Constraints

for k in Stores:
    prob += lpSum(Y[j][k] for j in DistributionCenter)+lpSum(Z[i][k] for i in Factory) >= Demand[k]
    
for j in DistributionCenter:
    prob += lpSum(X[i][j] for i in Factory) >= lpSum(Y[j][k] for k in Stores)
    
for i in Factory:
    prob += lpSum(X[i][j] for j in DistributionCenter)+lpSum(Z[i][k] for k in Stores) <= FCapacity[i]

### Solve problem

In [14]:
prob.solve()

1

In [15]:
print("The problem is ", LpStatus[prob.status])
print("The objective function value is ", value(prob.objective))
for v in prob.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)

The problem is  Optimal
The objective function value is  56100.0
x_Middle_Yard_Neptunus = 1500.0
x_Middle_Yard_Venus = 5500.0
x_Quarterage_Ceres = 3950.0
x_Quarterage_Mars = 1000.0
x_Quarterage_Neptunus = 5050.0
x_Tonoz_Mars = 3000.0
x_White_Kiosk_Ceres = 250.0
x_White_Kiosk_Minerva = 4750.0
